In [10]:
import pandas as pd
from data_reciever import datasets

In [11]:
def check_dataset(name, display_name=None):
    if display_name is None:
        display_name = name.replace('_', ' ').title()
    
    print(f"xxx {display_name.upper()} xxx")
    if name in datasets and datasets[name] is not None:
        print(datasets[name].head())
        print(f"Shape: {datasets[name].shape}")
    else:
        print("Dataset not loaded")
    print()

# Check all datasets
check_dataset('menu_ingredient', 'MENU INGREDIENT')
check_dataset('seasons', 'SEASONS')
check_dataset('sales_history', 'SALES HISTORY')
check_dataset('event', 'EVENT')
check_dataset('weekend', 'WEEKEND')

xxx MENU INGREDIENT xxx
   item_id                  name     category  price  \
0        1  Paneer Butter Masala  Main Course    280   
1        2       Chicken Biryani  Main Course    320   
2        3        Veg Fried Rice  Main Course    200   
3        4             Dal Tadka  Main Course    180   
4        5           Butter Naan        Bread     50   

                                         ingredients  
0  [{'name': 'Paneer', 'quantity': '150g'}, {'nam...  
1  [{'name': 'Basmati Rice', 'quantity': '150g'},...  
2  [{'name': 'Rice', 'quantity': '150g'}, {'name'...  
3  [{'name': 'Toor Dal', 'quantity': '120g'}, {'n...  
4  [{'name': 'Flour', 'quantity': '80g'}, {'name'...  
Shape: (30, 5)

xxx SEASONS xxx
Dataset not loaded

xxx SALES HISTORY xxx
  Customer_ID        Date                                      Items_Ordered  \
0    CUST1018  2024-10-01                          Egg Curry x1; Fish Fry x1   
1    CUST1021  2024-10-01  Caesar Salad x2; Paneer Butter Masala x2; Pane..

In [ ]:
# More concise approach
df_exploded = df.copy()
df_exploded['menu_ingredient'] = df_exploded['Items_Ordered'].str.split('; ')

# Explode the list and separate dish from quantity
df_exploded = df_exploded.explode('Items_Ordered')
df_exploded[['Dish', 'Quantity']] = df_exploded['Items_Ordered'].str.rsplit(' x', n=1, expand=True)
df_exploded['Quantity'] = df_exploded['Quantity'].astype(int)

# Drop the original column and keep the structured data
date_itemquantity_holdiay = df_exploded[['Date', 'Customer_ID', 'Dish', 'Quantity']].reset_index(drop=True)
print(date_itemquantity_holdiay)